In [101]:
from typing import List, Union

# Program Runner

In [102]:
instructions_to_params = {
    '01': 3,
    '02': 3,
    '03': 1,
    '04': 1,
    '05': 2,
    '06': 2,
    '07': 3,
    '08': 3,
    '99':0
}

In [103]:
def amplify(phase: int, amp_input: int, program_integers: List[str]) -> Union[int, None]:
    input_queue = [phase, amp_input]
    integers = program_integers.copy()
    i = 0
    op_code = integers[i]
    while op_code != '99':
        op, param_modes = op_code[-2:].zfill(2), op_code[:-2]
        length = instructions_to_params[op]
        #print('Op: ', op)
        #print('Param_modes: ', param_modes)
        #print('i: ', i)
        param_modes = [int(i) for i in param_modes.zfill(length)[::-1]]
        params = [int(param) for param in integers[i+1:i+1+length]]
        #print('Initial params: ', params)
        params = [int(integers[value]) if param_modes[pos] == 0 and pos != length-1 else value for pos, value in enumerate(params)]
        #print('Params: ', params)
        if op == '01':
            # Add - 3 params
            a, b, output_pos = params
            output = a + b
            integers[output_pos] = output
            i += length + 1
        elif op == '02':
            # Multiply
            a, b, output_pos = params
            output = a * b
            integers[output_pos] = output
            i += length + 1
        elif op == '03':
            # Read input
            output_pos = params[0]
            output, *input_queue = input_queue
            integers[output_pos] = output
            i += length + 1
        elif op == '04':
            # Print output
            output_pos = params[0]
            # Return as we are in an amplifier
            return integers[output_pos]
            i += length + 1
        elif op == '05':
            # Jump-if-true
            value, output_val = params
            if param_modes[-1] == 0:
                output_val = int(integers[output_val])
            if value > 0:
                i = output_val
            else:
                i += length + 1
        elif op == '06':
            # Jump-if-true
            value, output_val = params
            if param_modes[-1] == 0:
                output_val = int(integers[output_val])
            if value == 0:
                i = output_val
            else:
                i += length + 1
        elif op == '07':
            # Less than
            a, b, output_pos = params
            output = 1 if a < b else 0
            integers[output_pos] = output
            i += length + 1
        elif op == '08':
            # Equals
            a, b, output_pos = params
            output = 1 if a == b else 0
            integers[output_pos] = output
            i += length + 1
        else:
            raise Exception('Invalid op_code supplied: ', op_code)
        op_code = str(integers[i])
    return None

In [104]:
filename = 'input.txt'
#filename = 'test1.txt'
with open(filename) as f:
    initial_integers = [element for element in f.read().replace('/n', '').split(',')]

# Part 1

## Testing the program runner
```
test_phase = [4,3,2,1,0]
prev_output = 0
for amp, phase in enumerate(test_phase):
    print(f'Amplifier {amp}, Phase: {phase}, prev_output: {prev_output}')
    prev_output = amplify(phase, prev_output, initial_integers)
print(initial_integers)
prev_output
```

In [141]:
# Basically does itertools.product but I wanted to implement it myself
def make_permutations(num_lists: int, size: int, start: int = 0) -> List[List[int]]:
    list_range = range(start, start+size+1)
    lists = [[]]
    for _ in range(num_lists):
        lists = [l + [i] for i in list_range for l in lists]
    return lists

In [106]:
# Run through the whole amplifier chain
def amplify_chain(config: List[int]) -> int:
    prev_output = 0
    for phase in config:
        prev_output = amplify(phase, prev_output, initial_integers)
    return prev_output

In [107]:
acceptable_configs = [config for config in make_permutations(5,4) if sorted(config.copy()) == [0,1,2,3,4]]
parameter_values = [(''.join([str(i) for i in config]), amplify_chain(config)) for config in acceptable_configs]
max(parameter_values, key=lambda x: x[1])

('02431', 567045)

# Part 2

In [133]:
class Amplifier:
    def __init__(self, program_integers: List[str]):
        self.integers = program_integers.copy()
        self.signal = 0
        self.i = 0
        

    def amplify(self, input_queue: List[int]) -> Union[int, None]:
        #print('Initial i: ', self.i)
        #print('Input queue: ', input_queue)
        op_code = self.integers[self.i]
        while op_code != '99':
            op, param_modes = op_code[-2:].zfill(2), op_code[:-2]
            length = instructions_to_params[op]
            #print('Op: ', op)
            #print('Param_modes: ', param_modes)
            #print('i: ', self.i)
            param_modes = [int(m) for m in param_modes.zfill(length)[::-1]]
            params = [int(param) for param in self.integers[self.i+1:self.i+1+length]]
            #print('Initial params: ', params)
            params = [int(self.integers[value]) if param_modes[pos] == 0 and pos != length-1 else value for pos, value in enumerate(params)]
            #print('Params: ', params)
            if op == '01':
                # Add - 3 params
                a, b, output_pos = params
                output = a + b
                self.integers[output_pos] = output
                self.i += length + 1
            elif op == '02':
                # Multiply
                a, b, output_pos = params
                output = a * b
                self.integers[output_pos] = output
                self.i += length + 1
            elif op == '03':
                # Read input
                output_pos = params[0]
                output, *input_queue = input_queue
                self.integers[output_pos] = output
                self.i += length + 1
            elif op == '04':
                # Print output
                output_pos = params[0]
                # Return as we are in an amplifier
                self.signal = self.integers[output_pos]
                self.i += length + 1
                return self.signal
            elif op == '05':
                # Jump-if-true
                value, output_val = params
                if param_modes[-1] == 0:
                    output_val = int(self.integers[output_val])
                if value > 0:
                    self.i = output_val
                else:
                    self.i += length + 1
            elif op == '06':
                # Jump-if-true
                value, output_val = params
                if param_modes[-1] == 0:
                    output_val = int(self.integers[output_val])
                if value == 0:
                    self.i = output_val
                else:
                    self.i += length + 1
            elif op == '07':
                # Less than
                a, b, output_pos = params
                output = 1 if a < b else 0
                self.integers[output_pos] = output
                self.i += length + 1
            elif op == '08':
                # Equals
                a, b, output_pos = params
                output = 1 if a == b else 0
                self.integers[output_pos] = output
                self.i += length + 1
            else:
                raise Exception('Invalid op_code supplied: ', op_code)
            op_code = str(self.integers[self.i])
        return None

## Testing loop
```
test_program = [3,26,1001,26,-4,26,3,27,1002,27,2,27,1,27,26,27,4,27,1001,28,-1,28,1005,28,6,99,0,0,5]
test_program = [str(i) for i in test_program]
phase_config = [9,8,7,6,5]
target = 139629729
amps = [Amplifier(test_program) for _ in phase_config]
i = prev_output = 0
phased = False
while prev_output is not None:
    #print('Amplifier: ', i)
    input_queue = [prev_output] if phased else [phase_config[i], prev_output]
    prev_output = amps[i].amplify(input_queue)
    #print(prev_output)
    i = (i+1)%len(amps)
    if i == 0 and not phased:
        phased = True
print('Signal: ', amps[-1].signal)
```

In [136]:
# Run through the whole amplifier chain
def amplify_chain(config: List[int]) -> int:
    amps = [Amplifier(initial_integers) for _ in config]
    i = prev_output = 0
    phased = False
    while prev_output is not None:
        #print('Amplifier: ', i)
        input_queue = [prev_output] if phased else [config[i], prev_output]
        prev_output = amps[i].amplify(input_queue)
        #print(prev_output)
        i = (i+1)%len(amps)
        if i == 0 and not phased:
            phased = True
    return amps[-1].signal

In [143]:
acceptable_configs = [config for config in make_permutations(5,4,5) if sorted(config.copy()) == [5,6,7,8,9]]
parameter_values = [(''.join([str(i) for i in config]), amplify_chain(config)) for config in acceptable_configs]
max(parameter_values, key=lambda x: x[1])

('65789', 39016654)